In [ ]:
!pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git
!pip install telebot

In [ ]:
#@title Create model
from RealESRGAN import RealESRGAN
from PIL import Image
import numpy as np
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

model_scale = "4" #@param ["2", "4", "8"] {allow-input: false}

model = RealESRGAN(device, scale=int(model_scale))
model.load_weights(f'weights/RealESRGAN_x{model_scale}.pth')

In [ ]:
import os
from google.colab import files
from io import BytesIO
import telebot
from PIL import Image
import numpy as np

upload_folder = 'inputs'
result_folder = 'results'

bot = telebot.TeleBot('telegram_bot_token')

os.makedirs(upload_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

IMAGE_FORMATS = ('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')

MAX_DIMENSION = 5000  # Maximum dimension for the image

def process_image(image_bytes, image_name, chat_id):
    try:
        image = Image.open(BytesIO(image_bytes)).convert('RGB')

        # Resize the image if it exceeds the maximum dimension
        if max(image.width, image.height) > MAX_DIMENSION:
            ratio = MAX_DIMENSION / max(image.width, image.height)
            size = (int(image.width * ratio), int(image.height * ratio))
            image = image.resize(size)

        sr_image = model.predict(np.array(image))
    except Exception as err:
        print(f'Unable to process file {image_name}, skipping')
        return

    result_image_path = os.path.join('results/', 'result.jpg')
    os.makedirs(os.path.dirname(result_image_path), exist_ok=True)
    sr_image.save(result_image_path)

    # send the result image to the user who sent the original photo
    with open(result_image_path, 'rb') as f:
        bot.send_photo(chat_id=chat_id, photo=f)

    bot.send_message(chat_id=chat_id, text="Upscale complete!")

    print(f'Finished! Image saved to {result_image_path}')

@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    chat_id = message.chat.id
    file_info = bot.get_file(message.photo[-1].file_id)
    file_bytes = bot.download_file(file_info.file_path)
    
    if len(file_bytes) > 0 and file_bytes[:4] == b'\x1f\x8b\x08\x00':
        print('gzip format detected, skipping')
        return
        
    image_name = 'input.jpg'
    process_image(file_bytes, image_name, chat_id)

bot.polling() # start listening for incoming messages

Finished! Image saved to results/result.jpg
